# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with mongoimport --type json -d uk_food -c establisments --drop --jsonArray establishments.json


In [13]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [14]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [15]:
# confirm that our new database was created
print(mongo.list_database_names())



['admin', 'classDB', 'config', 'fruits_db', 'local', 'met', 'travel_db', 'uk_food']


In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our new database
print(uk_food_db.list_collection_names())





NameError: name 'uk_food_db' is not defined

In [ ]:
# review a document in the establishments collection
uk_food_db.establishments.find_one()



NameError: name 'uk_food_db' is not defined

In [ ]:
# assign the collection to a variable
establishments_list = uk_food_db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
Penang_Flavours = {
    "BusinessName":"Penang Flavours"
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}
# establishments_list.delete_one(Penang_Flavours)

In [ ]:
# Insert the new restaurant into the collection
establishments_list.insert_one(Penang_Flavours)



In [ ]:
# Check that the new restaurant was inserted
query = {"BusinessName":"Penang Flavours"}
query_results = establishments_list.find(query)

for result in query_results:
    print(result)


{'_id': ObjectId('646c2c7cfae657e3218d2928'),
 'address': {'city': 'London',
             'state': 'London',
             'street': '456 Elm St',
             'zipcode': 'EC1 1AA'},
 'name': 'Fancy Deli',
 'rating': 5}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
fields = {"BusinessTypeID", "BusinessType"}
sorts = [("BusinessTypeID", 1)]
limit = 5

query_results_ID = list(establishments_list.find(query,fields).sort(sorts).limit(limit))
pd.DataFrame(query_results_ID)

None


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:

Update the new restaurant with the BusinessTypeID you found.
# Update the new restaurant with the correct BusinessTypeID
newvalues = {'$set': {"BusinessTypeID": 1}}
establishments_list.update_one(query, newvalues)


{'BusinessTypeID': 7839,
 '_id': ObjectId('646c2c7cfae657e3218d2928'),
 'address': {'city': 'London',
             'state': 'London',
             'street': '456 Elm St',
             'zipcode': 'EC1 1AA'},
 'name': 'Fancy Deli',
 'rating': 5}


In [ ]:
# Confirm that the new restaurant was updated
query = {"BusinessName":"Penang Flavours"}
query_results = establishments_list.find(query)

for result in query_results:
    print(result)


{'BusinessTypeID': 7839,
 '_id': ObjectId('646c2c7cfae657e3218d2928'),
 'address': {'city': 'London',
             'state': 'London',
             'street': '456 Elm St',
             'zipcode': 'EC1 1AA'},
 'name': 'Fancy Deli',
 'rating': 5}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': {'$regex': 'Dover'}}
Dover_list = list(establishments_list.find(query))
print(len(Dover_list))

0


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {'LocalAuthorityName': {'$regex': 'Dover'}}
establishments_list.delete_many(query)

# # Confirm that the documents were deleted
# pprint(establishments.count_documents({"LocalAuthorityName": "Dover"}))



0


In [ ]:
# Check if any remaining documents include Dover
query = {'LocalAuthorityName': {'$regex': 'Dover'}}
Dover_list = list(establishments_list.find(query))
if len(Dover_list) == 0:
    print(f'The number of documents including Dover is {len(Dover_list)}.')
else:
    print(f'len(Dover_list) documents still include Dover, please delete them')

None


In [ ]:
# Check that other documents remain with 'find_one'
establishments_list.find_one()


{'BusinessTypeID': 7839,
 '_id': ObjectId('646c2c7cfae657e3218d2928'),
 'address': {'city': 'London',
             'state': 'London',
             'street': '456 Elm St',
             'zipcode': 'EC1 1AA'},
 'name': 'Fancy Deli',
 'rating': 5}


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_one({"name": "Fancy Deli"}, {"$set": {"longitude": Decimal('51.52255')}})
establishments.update_one({"name": "Fancy Deli"}, {"$set": {"latitude": Decimal('-0.09599')}})

# # Change the data type from String to Decimal for longitude
# establishments_list.update_many({}, [{'$set':{ "geocode.longitude" : {'$toDouble': "$geocode.longitude"}}}])
# Change the data type from String to Decimal for latitude
# establishments_list.update_many({}, [{'$set':{ "geocode.latitude" : {'$toDouble': "$geocode.latitude"}}}])
    


NameError: name 'Decimal' is not defined

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set':{ "RatingValue" : {'$toInt': "$RatingValue"}}}])




In [ ]:
# Check that the coordinates and rating value are now numbers
query = {"name": "Fancy Deli"}
query_results = establishments_list.find(query)

